In [2]:
import json
import time
from time import sleep
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from pandas import *

In [3]:
#가게 이름, 가게 유형, 주소, 대표 메뉴 1개, 메뉴 설명, 리뷰 5개
def search_food(key):
    
    food_list = []
    
    url = 'https://map.naver.com/v5/search'
    driver = webdriver.Chrome()  # 드라이버 경로
    # driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
    driver.get(url)
    key_word = key  # 검색어

    # css 찾을때 까지 10초대기
    def time_wait(num, code):
        try:
            wait = WebDriverWait(driver, num).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, code)))
        except:
            print(code, '태그를 찾지 못하였습니다.')
            driver.quit()
        return wait

    # frame 변경 메소드
    def switch_frame(frame):
        driver.switch_to.default_content()  # frame 초기화"
        driver.switch_to.frame(frame)  # frame 변경

    # 페이지 다운
    def page_down(num):
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.click()
        for i in range(num):
            body.send_keys(Keys.PAGE_DOWN)

    # css를 찾을때 까지 10초 대기
    time_wait(10, 'div.input_box > input.input_search')

    # (1) 검색창 찾기
    search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

    sleep(5)
    
    url_str = driver.current_url
    if url_str.find("isCorrectAnswer") == -1:
        print(url_str)
        print("결과 여러 개 존재(지점 또는 근접 지역)")
        
        search = driver.find_elements(By.ID, "searchIframe")
        # frame 변경
        switch_frame('searchIframe')
        sleep(2)
    
        #검색 결과 없음
        place_none = driver.find_elements(By.CSS_SELECTOR, '.FYvSc')
        
        for i in range(len(place_none)):
            if place_none[i].text == "조건에 맞는 업체가 없습니다.":
                tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
                        '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
                print("가게 없음")
                return tmp
        
        tmp_shop = driver.find_elements(By.CSS_SELECTOR, '.YwYLL')
        if tmp_shop == []:
            tmp_shop = driver.find_elements(By.CSS_SELECTOR, '.place_bluelink')
        tmp_shop[0].click()
        
        sleep(5)
    
    else:
        print(url_str)
        print("결과 한 개")
    
    # (2) frame 변경
    switch_frame('entryIframe')
    #page_down(40)
    sleep(2)

    # 시작시간
    start = time.time()
    print('\n[크롤링 시작...]')
    
    shop_name = []
    shop_type = []
    shop_address = []
    menu_name = []
    menu_story = []
    review_1 = []
    review_2 = []
    review_3 = []
    review_4 = []
    review_5 = []
    

    #가게 이름
    names = driver.find_elements(By.CSS_SELECTOR, '.GHAhO')  # (3) 장소명
    shop_name.append(names[0].text)
    print(names[0].text)
    
    #가게 타입
    types = driver.find_elements(By.CSS_SELECTOR, '.lnJFt')  # (4) 장소 유형
    shop_type.append(types[0].text)
    print(types[0].text)
    
    #가게 주소
    address = driver.find_elements(By.CSS_SELECTOR, '.LDgIH') # (5) 주소
    shop_address.append(address[0].text)
    print(address[0].text)
    
    #버튼 정의
    home_list = driver.find_elements(By.CSS_SELECTOR, ".veBoZ") # (6) 리뷰
    
    #잠시 대기
    sleep(3)
    
    #배달 포장 여부
    order = driver.find_elements(By.XPATH, '/html/body/div[3]/div/div/div/div[2]/div[4]/div')
    order2 = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[4]/div/span/a')
    
    #print(order2)
    #print(order[0].text)
    if order[0].text == "주문준비중" or order[0].text == "주문\n배달" or order[0].text == "예약\n주문준비중" or order[0].text == "예약\n주문":
        order_result = 1 #배달 포장 가능
        print("주문 유형 1")
        
    elif (len(order2) != 0) and (order2[0].text == "주문"):
        order_result = 1 #배달 포장 가능
        print("주문 유형 2")
        
    else:
        order_result = 0 #배달 포장 불가능
        print("주문 유형 3")
    
    #메뉴 버튼 클릭
    for k in range(len(home_list)):
        if home_list[k].text == "메뉴":
            #print(home_list[k].text)
            home_list[k].click()
            sleep(3)
            break
        else:
            continue
    
    #배달 불가
    if order_result == 0:
        #상위 메뉴 1개 이름
        #print(menu_name)
        
        menu_btn_1 = driver.find_elements(By.CSS_SELECTOR, ".lPzHi") # (6-1) 메뉴
        
        if menu_btn_1 == []:
            menu_name.append("")
            print("대표 메뉴 없음")
        else: 
            menu_name.append(menu_btn_1[0].text)
            print(menu_btn_1[0].text)

        #메뉴 설명
        menu_btn_2 = driver.find_elements(By.CSS_SELECTOR, ".kPogF") # (6-2) 메뉴 설명
        if menu_btn_2 == []:
            menu_story.append("")
            print("메뉴 설명 없음")
            
        else:
            menu_story.append(menu_btn_2[0].text)
            print(menu_btn_2[0].text)
        
        sleep(3)
        
        
        #리뷰 버튼 클릭
        #메뉴 버튼 클릭
        for k in range(len(home_list)):
            if home_list[k].text == "리뷰":
                home_list[k].click()
                sleep(3)
                break
            else:
                continue
        
    #배달 가능
    else:
        sleep(3)
        #상위 메뉴 1개 이름
        #box = driver.find_elements(By.CSS_SELECTOR, ".info_detail")
        
        menu_btn_1 = driver.find_elements(By.CSS_SELECTOR, ".tit") # (6-1) 메뉴
        #print(menu_btn_1)
        menu_name.append(menu_btn_1[0].text)
        print(menu_btn_1[0].text)
        
        #메뉴 설명
        menu_btn_2 = driver.find_elements(By.CSS_SELECTOR, ".detail_txt") # (6-2) 메뉴 설명
        if menu_btn_2 == []:
            menu_story.append("")
            print("메뉴 설명 없음")
            
        else:
            menu_story.append(menu_btn_2[0].text)
            print(menu_btn_2[0].text)
        
        sleep(3)
        
        #리뷰 버튼 클릭
        #메뉴 버튼 클릭
        home_review_list = driver.find_elements(By.CSS_SELECTOR, '.txt')
        for k in range(len(home_review_list)):
            if home_review_list[k].text == "리뷰":
                home_review_list[k].click()
                sleep(3)
                break
            else:
                continue
    
    #리뷰 찾기
    review_btn = driver.find_elements(By.CSS_SELECTOR, ".t3JSf") # (7) 리뷰
    
    #이런 점이 좋았어요
    review_love = driver.find_elements(By.CSS_SELECTOR, ".place_section_header_title")


    
    review_love_yes = 0
    for i in range(len(review_love)):
        if review_love[i].text == "이런 점이 좋았어요\n안내":
            review_love_yes = 1
            print("리뷰 존재")
            break
        else:
             continue
             
    
    if review_love_yes == 0:
        tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
                    '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
    
    else:
        
        num_10_none = driver.find_elements(By.CSS_SELECTOR, ".zXXsw")
        
        if num_10_none != []:
            if num_10_none[0].text == "10명 이상 참여하면\n방문 키워드 통계가 공개됩니다!":
                print("리뷰 10명 이하 참여")
                tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
                        '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
            
        #리뷰 10명 이하 참여
        #if num_10_none[0].text == "10명 이상 참여하면\n방문 키워드 통계가 공개됩니다!":
        #    tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-",
        #            '메뉴 이름':"-", '메뉴 설명' : "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
        
        #리뷰 10명 이상 참여
        else:
            #리뷰 1
            real_review = review_btn[0].text.split('"')
            review_1.append(real_review[1])
            print(real_review[1])
            #리뷰 2
            real_review = review_btn[1].text.split('"')
            review_2.append(real_review[1])
            print(real_review[1])
            #리뷰 3
            real_review = review_btn[2].text.split('"')
            review_3.append(real_review[1])
            print(real_review[1])
            #리뷰 4
            real_review = review_btn[3].text.split('"')
            review_4.append(real_review[1])
            print(real_review[1])
            #리뷰 5
            real_review = review_btn[4].text.split('"')
            review_5.append(real_review[1])
            print(real_review[1])
        
            tmp = DataFrame({'가게명': shop_name, '가게 유형' :shop_type, '가게 주소': shop_address,
                            '메뉴 이름':menu_name, '메뉴 설명' : menu_story, "리뷰1": review_1, "리뷰2": review_2, "리뷰3": review_3, "리뷰4": review_4,"리뷰5": review_5})

    print(f'{shop_name} ...완료')

    sleep(1)

    print('[데이터 수집 완료]\n소요 시간 :', time.time() - start, "\n")
    driver.quit()  # 작업이 끝나면 창을 닫는다.
    
    return tmp

In [260]:

regions = ["종로구", "중구", "용산구", "성동구", "광진구", "동대문구", "중랑구", "성북구", "강북구", "도봉구", "노원구", "은평구", "서대문구", "마포구", "양천구", 
           "강서구", "구로구", "금천구", "영등포구", "동작구", "관악구", "서초구", "강남구", "송파구", "강동구"]

#len(regions)
for i in range(24, len(regions)):
    tmp = DataFrame(columns = ['가게명', '가게 유형', '가게 주소', '메뉴 이름', '메뉴 설명', '리뷰1', '리뷰2', '리뷰3', '리뷰4', '리뷰5'])
    filename = "./out_cafe/out_" + str(i) + "_cafe.csv"
    df = read_csv(filename, header = None)
    df.columns = ['가게명', '별점']
    
    #43, len(df)
    for j in range(62, len(df)):
        tp = df.loc[j]['가게명']
        key = regions[i] + " " + tp
        print(str(j) + "번째 가게")
        
        data = search_food(key)
        
        result2 = concat([tmp, data], ignore_index = True)
        tmp = result2

    #파일 저장
    filename = "./out_cafe_review/out_" + str(i) + "_cafe.csv"
    tmp.to_csv(filename, header = None, index = False)
    print(filename + "완료")


62번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EA%B0%95%EB%8F%99%EA%B5%AC%20%EC%BC%80%ED%82%A4%EB%8F%84%EB%A6%AC/place/1925928850?c=15.00,0,0,0,dh&isCorrectAnswer=true
결과 한 개

[크롤링 시작...]
케키도리
케이크전문
서울 강동구 구천면로18길 28 1층
주문 유형 2
도시락 케이크대표
속크림:생크림 겉크림:생크림&크림치즈
리뷰 존재
리뷰 10명 이하 참여
['케키도리'] ...완료
[데이터 수집 완료]
소요 시간 : 19.71349811553955 

63번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EA%B0%95%EB%8F%99%EA%B5%AC%20%EC%BB%A4%ED%94%BC%EB%B9%88%20%ED%98%84%EB%8C%80%EC%9E%90%EB%8F%99%EC%B0%A8%EC%84%B1%EB%82%B4%EC%A0%90?c=15.00,0,0,0,dh
결과 여러 개 존재(지점 또는 근접 지역)

[크롤링 시작...]
커피빈 현대자동차성내점
카페
서울 강동구 천호대로 1096
주문 유형 2
얼그레이 크림 초콜릿 라떼 (S)N
얼그레이의 향긋한 베르가못 향과 네덜란드 코코아의 깊고 진한 초콜릿의 풍미가 조화로운 음료. 얼그레이가 콕콕 박혀있는 크림이 포인트!
리뷰 존재
커피가 맛있어요
음료가 맛있어요
대화하기 좋아요
친절해요
매장이 청결해요
['커피빈 현대자동차성내점'] ...완료
[데이터 수집 완료]
소요 시간 : 19.24469780921936 

64번째 가게


...

https://map.naver.com/p/search/%EA%B0%95%EB%8F%99%EA%B5%AC%20%EC%8A%A4%EC%9C%97%ED%8B%B0%ED%81%B4%EC%8A%A4%20%EB%B0%80%ED%81%AC%ED%8B%B0&%EB%A7%88%EC%B9%B4%EB%A1%B1/place/1749623360?c=15.00,0,0,0,dh&isCorrectAnswer=true
결과 한 개

[크롤링 시작...]
스윗티클스 밀크티&마카롱
카페,디저트
서울 강동구 성내로3길 36 스윗티클스
주문 유형 1
(이*윤 님)개인결제창
12구 X 100세트
리뷰 존재
디저트가 맛있어요
친절해요
특별한 메뉴가 있어요
매장이 청결해요
커피가 맛있어요
['스윗티클스 밀크티&마카롱'] ...완료
[데이터 수집 완료]
소요 시간 : 18.30885338783264 

76번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EA%B0%95%EB%8F%99%EA%B5%AC%20%EC%95%88%EB%8B%A4%EB%AF%B8%EB%A1%9C%20%ED%8C%AC%EC%BC%80%EC%9D%B4%ED%81%AC%20%EC%B2%9C%ED%98%B8%EC%A0%90/place/1268106213?c=15.00,0,0,0,dh&isCorrectAnswer=true
결과 한 개

[크롤링 시작...]
안다미로 팬케이크 천호
카페,디저트
서울 강동구 천호옛14길 40 2층
주문 유형 3
안다미로 수플레
제철과일과 함께 즐기는 안다미로의 시그니쳐 수플레 팬케이크
리뷰 존재
디저트가 맛있어요
인테리어가 멋져요
커피가 맛있어요
음료가 맛있어요
대화하기 좋아요
['안다미로 팬케이크 천호'] ...완료
[데이터 수집 완료]
소요 시간 : 13.424135446548462 

77번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EA%B0%95%EB%8F%99%EA%B5%AC%20%EC%B9%B4%ED%8E%98%20%EB%A7%A5%EA%B7%B8%EB%A0%88%EA%B3%A0%EB%A5%B4?c=15.00,0,0,0,dh
결과 여러 개 존재(지점 또는 근접 지역)
가게 없음
./out_cafe_review/out_24_cafe.csv완료


In [256]:
#지금 번호 확인
print(i)
print(j)

24
61


In [247]:
# # 합쳐야 할 때 맨 처음 한 번만 실행 
# 지역 4 건너 뜀
ex = tmp.copy()
len(ex)
#ex

18

In [261]:
ex2 = tmp.copy()
len(ex2)

16

In [262]:
import pandas as pd
tmp_df = pd.concat([ex,ex2], axis=0, ignore_index=True)
#tmp_df[-3:]
len(tmp_df)

74

In [263]:
ex = tmp_df.copy()
len(ex)

74

In [264]:
#파일 저장
filename = "./out_cafe_review/out_" + str(24) + "_cafe.csv"
ex.to_csv(filename, header = None, index = False)
print(filename + "완료")

./out_cafe_review/out_24_cafe.csv완료
